## Проверка данных Цифромеда и прочих

In [1]:
import pandas as pd
import numpy as np
import os
import sys

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
# файлы берём непосредственно из хранилища нового бота посещений
e_path = "C:/work/new_bot/new/e"
s_path = "C:/work/new_bot/new/s"
dm_path = "C:/work/new_bot/new/dm"
vis_reg_ref_path = "C:/work/new_bot/new/vis_reg_ref.csv"
e_file_list = os.listdir(e_path)
s_file_list = os.listdir(s_path)
dm_file_last = os.listdir(dm_path + "/" + max(os.listdir(dm_path)))
# выгрузки ПОС
pos_path = "C:/work/pos_bot/POS"
pos_dir_last = os.listdir(pos_path + "/" + max(os.listdir(pos_path)))
pos_reg_ref_path = "/".join(pos_path.split("/")[:-1]) + "/pos_reg_ref.csv"

# типы ошибок
t_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
    "Время уже занято другим пациентом",
    "Услуга посмертно не оказывается",
    "Время начала приема уже прошло",
]
tdb_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
]
b_list = [
    "Пациент уже записан на это время к другому специалисту",
    "Пациент уже записан к специалисту в этот день",
    "Выбранное время доступно только для записи пациентов определенного возраста",
]
o_error_list = [
    "Значение не найдено",
]
r_error_list = [
    "Ошибка «СМЭВ-timeout»",
    "Внутренняя ошибка концентратора при создании записи",
    "Ошибка во входных параметрах при записи на прием",
]
any_error_list = [
    "Ошибка доставки формы при записи на прием",
    "Истекло время сессии пользователя",
    "Ошибка соответствия ответа запросу МИС",
]
draft_list = ["Ошибка отсутсвует"]
# словарь списков типов ошибок
e_type_dict = {
    "t": t_error_list,
    "tdb": tdb_error_list,
    "b": b_list,
    "o": o_error_list,
    "r": r_error_list,
    "any": any_error_list,
    "draft": draft_list,
}


def error_table(df):
    """функция принимает только (!) df_error_yesterday или df_error_today из возвращает df ошибок"""
    df_tmp_o = df[df["final_state"].isin(o_error_list)]
    df_tmp_o["type"] = "орг. дашборд"
    df_tmp_t = df[df["final_state"].isin(t_error_list)]
    df_tmp_t["type"] = "технические всего"
    df_tmp_tdb = df[df["final_state"].isin(tdb_error_list)]
    df_tmp_tdb["type"] = "технические дашборд"
    df_tmp_b = df[df["final_state"].isin(b_list)]
    df_tmp_b["type"] = "бизнес-правила"
    df_tmp_r = df[df["final_state"].isin(r_error_list)]
    df_tmp_r["type"] = "фэр дашборд"
    df_tmp_any = df[df["final_state"].isin(any_error_list)]
    df_tmp_any["type"] = "прочая"
    df_tmp_draft = df[df["final_state"].isin(draft_list)]
    df_tmp_draft["type"] = "прервано дашборд"
    df_error_type = pd.concat(
        [df_tmp_o, df_tmp_t, df_tmp_tdb, df_tmp_b, df_tmp_r, df_tmp_any, df_tmp_draft]
    ).reset_index(drop=True)
    df_error_type = (
        df_error_type.pivot_table(
            index="Название субъекта РФ",
            columns="type",
            values="session_id",
            aggfunc="count",
        )
        .reset_index()
        .fillna(0)
        .rename_axis(None, axis=1)
    )
    df_error_type = df_error_type.astype(
        {
            "бизнес-правила": "int",
            "орг. дашборд": "int",
            "прервано дашборд": "int",
            "технические всего": "int",
            "технические дашборд": "int",
            "фэр дашборд": "int",
        }
    )
    return df_error_type

### Ошибки за две последние даты

In [3]:
%%time
# ошибки записи (два последние файла)
m=0
for file_name in e_file_list[-2:]:
    if m == 0:
        file_name_for_error_count_yesterday = file_name
    else:
        file_name_for_error_count_today = file_name
    df_tmp = pd.read_csv(e_path + "/" + file_name, sep=";")
    print(f"\nФайл {file_name}:")
    print(f"\t\tвсего строк:\t\t\t{df_tmp.shape[0]}")
    cnt_c = "верно" if df_tmp.shape[1] == 13 else "ошибка"
    print(f"\t\tвсего столбцов:\t\t\t{df_tmp.shape[1]} ({cnt_c})")
    print("\tПропуски в столбцах, где их быть не должно:")
    df_nonan = df_tmp[
        [
            "session_id",
            "Название субъекта РФ",
            "step_id",
            "step_name",
            "error_code",
            "final_state",
            "create_ts",
            "Тип записи",
            "referral_flg",
        ]
    ]
    n = 0
    for column in df_nonan.columns:
        if df_nonan[column].isnull().sum() != 0:
            print(
                f"\t\tстолбец '{column}':\t\t {df_nonan[column].isnull().sum()} ({round(df_nonan[column].isnull().sum()*100/df_nonan.shape[0], 2)} %)"
            )
            n += 1
    if n == 0:
        print("\t\tотсутствуют")
    print(
        "\tОшибки данных (количество и доля от числа строк на соответствующем этапе):"
    )
    # число строк на соответствующих этапах
    len_ValidatePerson = df_tmp[df_tmp["step_name"] == "ValidatePerson"].shape[0]
    len_MO = df_tmp[df_tmp["step_name"] == "MO"].shape[0]
    len_ServiceOrSpecs = df_tmp[df_tmp["step_name"] == "ServiceOrSpecs"].shape[0]
    len_Slot = df_tmp[df_tmp["step_name"] == "Slot"].shape[0]
    len_Book = df_tmp[df_tmp["step_name"] == "Book"].shape[0]

    """
    На этапе валидации пациента
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ValidatePerson = df_tmp[
        (df_tmp["step_name"] == "ValidatePerson")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора МО
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_MO = df_tmp[
        (df_tmp["step_name"] == "MO")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора специальности врача
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ServiceOrSpecs = df_tmp[
        (df_tmp["step_name"] == "ServiceOrSpecs")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """"
    На этапе выбора слота
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ нет данных о специальтности врача
    ИЛИ есть данные о времени слота
    """
    df_Slot = df_tmp[
        (df_tmp["step_name"] == "Slot")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе записи
    нет любых данных (ИЛИ)
    """
    df_Book = df_tmp[
        (df_tmp["step_name"] == "Book")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | ~df_tmp["slot_ts"].notnull()
        )
    ]

    if df_ValidatePerson.shape[0] >= 0:
        print(
            f"\t\tValidatePerson_uncorrected:\t{df_ValidatePerson.shape[0]}\t({round(df_ValidatePerson.shape[0]*100/len_ValidatePerson, 2)} %)"
        )
    if df_MO.shape[0] >= 0:
        print(
            f"\t\tMO_uncorrected:\t\t\t{df_MO.shape[0]}\t({round(df_MO.shape[0]*100/len_MO, 2)} %)"
        )
    if df_ServiceOrSpecs.shape[0] >= 0:
        print(
            f"\t\tServiceOrSpecs_uncorrected:\t{df_ServiceOrSpecs.shape[0]}\t({round(df_ServiceOrSpecs.shape[0]*100/len_ServiceOrSpecs, 2)} %)"
        )
    if df_Slot.shape[0] >= 0:
        print(
            f"\t\tSlot_uncorrected:\t\t{df_Slot.shape[0]}\t({round(df_Slot.shape[0]*100/len_Slot, 2)} %)"
        )
    if df_Book.shape[0] >= 0:
        print(
            f"\t\tBook_uncorrected:\t\t{df_Book.shape[0]}\t({round(df_Book.shape[0]*100/len_Book, 2)} %)"
        )
    if (
        df_ValidatePerson.shape[0]
        + df_MO.shape[0]
        + df_ServiceOrSpecs.shape[0]
        + df_Slot.shape[0]
        + df_Book.shape[0]
        == 0
    ):
        print("\t\tотсутствуют")

    # субъекты, которых нет на дашборде
    df_tmp_3 = df_tmp[df_tmp['Тип записи'] == 3]

    # данные для сравнительного расчёта
    if m == 0:
        df_error_yesterday = df_tmp_3.copy()
    else:
        df_error_today = df_tmp_3.copy()
    
    df_nosub = df_tmp_3[
        (df_tmp["Название субъекта РФ"] == "Федеральное медико-биологическое агентство")
        | (df_tmp["Название субъекта РФ"] == "г. Байконур")
    ][["Название субъекта РФ", "final_state"]]
    df_nosub["value"] = 1
    # ошибки, которых нет на дашборде
    df_nosub_e = (
        df_nosub.groupby(["Название субъекта РФ", "final_state"])
        .sum()
        .reset_index()[["final_state", "value"]]
    )
    no_db_e_dict = {}
    for key, value in e_type_dict.items():
        no_db_e_dict[str(key + "_ndb")] = df_nosub_e[
            df_nosub_e["final_state"].isin(value)
        ]["value"].sum()

    df_errors = pd.DataFrame(
        df_tmp[df_tmp["Тип записи"] == 3].groupby("final_state")["session_id"].count()
    ).reset_index()
    print('\tКоличество ошибок записи на приём (тип записи "К врачу"; рассылка (дашборд)):')
    o_e = df_errors[df_errors["final_state"].isin(o_error_list)]["session_id"].sum()
    print(f'\t\tорганизационные ошибки:\t\t{o_e}\t({o_e - no_db_e_dict["o_ndb"]})')
    t_e = df_errors[df_errors["final_state"].isin(t_error_list)]["session_id"].sum()
    t_e_db = (
        df_errors[df_errors["final_state"].isin(tdb_error_list)]["session_id"].sum()
        - no_db_e_dict["tdb_ndb"] # было t_ndb
    )
    print(f"\t\tтехнические ошибки:\t\t{t_e}\t({t_e_db})")
    r_e = df_errors[df_errors["final_state"].isin(r_error_list)]["session_id"].sum()
    print(f'\t\tошибки ФЭР:\t\t\t{r_e}\t({r_e - no_db_e_dict["r_ndb"]})')
    any_e = df_errors[df_errors["final_state"].isin(any_error_list)]["session_id"].sum()
    print(f'\t\tпрочие ошибки:\t\t\t{any_e}\t({any_e - no_db_e_dict["any_ndb"]})')
    draft = df_errors[df_errors["final_state"].isin(draft_list)]["session_id"].sum()
    print(f'\t\tчерновики:\t\t\t{draft}\t({draft - no_db_e_dict["draft_ndb"]})')
    b = df_errors[df_errors["final_state"].isin(b_list)]["session_id"].sum()
    print(f'\t\tбизнес-правила:\t\t\t{b}\t({b - no_db_e_dict["b_ndb"]})')
    print("\tСубъекты, не отображаемые на дашборде:")
    if df_nosub.empty:
        print("\t\tотсутствуют")
    else:
        print(
            df_nosub.groupby(["Название субъекта РФ", "final_state"])
            .sum()
            .reset_index()
            .to_string(index=False)
        )
    m+=1
print("-----------------------------------------------------------------------------------------------")
print(f"Вчерашний файл:\t\t{file_name_for_error_count_yesterday}")
print(f"Актуальный файл:\t{file_name_for_error_count_today}")
print("-----------------------------------------------------------------------------------------------")
    # break


Файл 27.01.2024.csv:
		всего строк:			223620
		всего столбцов:			13 (верно)
	Пропуски в столбцах, где их быть не должно:
		отсутствуют
	Ошибки данных (количество и доля от числа строк на соответствующем этапе):
		ValidatePerson_uncorrected:	0	(0.0 %)
		MO_uncorrected:			0	(0.0 %)
		ServiceOrSpecs_uncorrected:	3066	(7.15 %)
		Slot_uncorrected:		1760	(2.95 %)
		Book_uncorrected:		54	(0.91 %)
	Количество ошибок записи на приём (тип записи "К врачу"; рассылка (дашборд)):
		организационные ошибки:		17402	(17389)
		технические ошибки:		23008	(22437)
		ошибки ФЭР:			10994	(10993)
		прочие ошибки:			0	(0)
		черновики:			166420	(166359)
		бизнес-правила:			4024	(4022)
	Субъекты, не отображаемые на дашборде:
                      Название субъекта РФ                                      final_state  value
Федеральное медико-биологическое агентство                              Значение не найдено     13
Федеральное медико-биологическое агентство Ошибка во входных параметрах при записи на прием  

In [19]:
no_db_e_dict

{'t_ndb': 6,
 'tdb_ndb': 5,
 'b_ndb': 5,
 'o_ndb': 79,
 'r_ndb': 10,
 'any_ndb': 0,
 'draft_ndb': 255}

#### Ошибки по регионам на последнюю дату

In [4]:
print(f"Файл: {file_name_for_error_count_today}")
error_table(df_error_today)

Файл: 28.01.2024.csv


,Название субъекта РФ,бизнес-правила,орг. дашборд,прервано дашборд,технические всего,технические дашборд,фэр дашборд
0,Алтайский край,5,56,9676,93,93,338
1,Амурская область,39,2,1074,1381,1379,10
2,Архангельская область,0,0,0,14115,14115,4
3,Астраханская область,5,60,181,46,46,8
4,Белгородская область,35,25,496,5,5,395
...,...,...,...,...,...,...,...
81,Ямало-Ненецкий автономный округ,5,101,156,3,3,2
82,Ярославская область,28,603,1740,152,151,284
83,г. Москва,0,406,5974,5294,5294,219
84,г. Санкт-Петербург,64,778,6627,487,479,673


#### Сравнение ошибок "вчера-сегодня"

In [6]:
df_e_y = error_table(df_error_yesterday)
df_e_t = error_table(df_error_today)
df_e_yt = df_e_t.merge(df_e_y, on="Название субъекта РФ", how="outer").fillna(0)
df_e_yt = df_e_yt.rename(
    columns={
        "бизнес-правила_x": "бп отч",
        "орг. дашборд_x": "оДБ отч",
        "прервано дашборд_x": "чДБ отч",
        "технические всего_x": "т отч",
        "технические дашборд_x": "тДБ отч",
        "фэр дашборд_x": "фДБ отч",
        "бизнес-правила_y": "бп прш",
        "орг. дашборд_y": "оДБ прш",
        "прервано дашборд_y": "чДБ прш",
        "технические всего_y": "т прш",
        "технические дашборд_y": "тДБ прш",
        "фэр дашборд_y": "фДБ прш",
    }
)
df_e_yt = df_e_yt.astype(
    {
        "бп отч": "int",
        "оДБ отч": "int",
        "чДБ отч": "int",
        "т отч": "int",
        "тДБ отч": "int",
        "фДБ отч": "int",
        "бп прш": "int",
        "оДБ прш": "int",
        "чДБ прш": "int",
        "т прш": "int",
        "тДБ прш": "int",
        "фДБ прш": "int",
    }
)
df_e_yt = df_e_yt[
    [
        "Название субъекта РФ",
        "бп отч",
        "бп прш",
        "оДБ отч",
        "оДБ прш",
        "чДБ отч",
        "чДБ прш",
        "т отч",
        "т прш",
        "тДБ отч",
        "тДБ прш",
        "фДБ отч",
        "фДБ прш",
    ]
]
df_e_yt.insert(3, "Δбп", df_e_yt["бп отч"] - df_e_yt["бп прш"])
df_e_yt.insert(6, "ΔоДБ", df_e_yt["оДБ отч"] - df_e_yt["оДБ прш"])
df_e_yt.insert(9, "ΔчДБ", df_e_yt["чДБ отч"] - df_e_yt["чДБ прш"])
df_e_yt.insert(12, "Δт", df_e_yt["т отч"] - df_e_yt["т прш"])
df_e_yt.insert(15, "ΔтДБ", df_e_yt["тДБ отч"] - df_e_yt["тДБ прш"])
df_e_yt["ΔфДБ"] = df_e_yt["фДБ отч"] - df_e_yt["фДБ прш"]

In [7]:
df_e_yt

,Название субъекта РФ,бп отч,бп прш,Δбп,оДБ отч,оДБ прш,ΔоДБ,чДБ отч,чДБ прш,ΔчДБ,т отч,т прш,Δт,тДБ отч,тДБ прш,ΔтДБ,фДБ отч,фДБ прш,ΔфДБ
0,Алтайский край,5,2,3,56,9,47,9676,7273,2403,93,52,41,93,52,41,338,243,95
1,Амурская область,39,42,-3,2,0,2,1074,1037,37,1381,52,1329,1379,46,1333,10,11,-1
2,Архангельская область,0,11,-11,0,43,-43,0,1389,-1389,14115,7011,7104,14115,7006,7109,4,60,-56
3,Астраханская область,5,6,-1,60,48,12,181,141,40,46,5,41,46,5,41,8,5,3
4,Белгородская область,35,17,18,25,23,2,496,444,52,5,20,-15,5,20,-15,395,440,-45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Ямало-Ненецкий автономный округ,5,68,-63,101,74,27,156,168,-12,3,1,2,3,1,2,2,3,-1
82,Ярославская область,28,45,-17,603,513,90,1740,1327,413,152,416,-264,151,414,-263,284,230,54
83,г. Москва,0,0,0,406,400,6,5974,6273,-299,5294,2919,2375,5294,2919,2375,219,52,167
84,г. Санкт-Петербург,64,39,25,778,804,-26,6627,5660,967,487,227,260,479,227,252,673,667,6


### Успехи за две последние даты

In [9]:
%%time
# успешные записи (два последние файла)
for file_name in s_file_list[-2:]:
    df_tmp_s = pd.read_csv(s_path + "/" + file_name, sep=";")
    print(f"\nФайл {file_name}:")
    print(f"\t\tвсего строк:\t\t{df_tmp_s.shape[0]}")
    cnt_s = "верно" if df_tmp_s.shape[1] == 9 else "ошибка"
    print(f"\t\tвсего столбцов:\t\t{df_tmp_s.shape[1]} ({cnt_s})")
    print("\tПропуски данных (всего (доля от общего числа строк)):")
    # пропуски данных
    missing_s_dict = {}
    for column in df_tmp_s.columns:
        missing_s_dict[column] = [
            df_tmp_s[column].isnull().sum(),
            round(df_tmp_s[column].isnull().sum() * 100 / df_tmp_s.shape[0], 2),
        ]
    print(f'\t\tДата:\t\t\t{missing_s_dict["Дата"][0]}\t({missing_s_dict["Дата"][1]} %)')
    print(
        f'\t\tsession_id:\t\t{missing_s_dict["session_id"][0]}\t({missing_s_dict["session_id"][1]} %)'
    )
    print(
        f'\t\tНазвание субъекта РФ:\t{missing_s_dict["Название субъекта РФ"][0]}\t({missing_s_dict["Название субъекта РФ"][1]} %)'
    )
    print(
        f'\t\tsp_oid:\t\t\t{missing_s_dict["sp_oid"][0]}\t({missing_s_dict["sp_oid"][1]} %)'
    )
    print(
        f'\t\tsp_name:\t\t{missing_s_dict["sp_name"][0]}\t({missing_s_dict["sp_name"][1]} %)'
    )
    print(
        f'\t\tpost_name:\t\t{missing_s_dict["post_name"][0]}\t({missing_s_dict["post_name"][1]} %)'
    )
    print(
        f'\t\tslot_ts:\t\t{missing_s_dict["slot_ts"][0]}\t({missing_s_dict["slot_ts"][1]} %)'
    )
    print(
        f'\t\tcreate_ts:\t\t{missing_s_dict["create_ts"][0]}\t({missing_s_dict["create_ts"][1]} %)'
    )
    print(
        f'\t\tТип записи:\t\t{missing_s_dict["Тип записи"][0]}\t({missing_s_dict["Тип записи"][1]} %)'
    )
    # субъекты, которых нет на дашборде
    df_nosub_s = df_tmp_s[
        (df_tmp_s["Название субъекта РФ"] == "Федеральное медико-биологическое агентство")
        | (df_tmp_s["Название субъекта РФ"] == "г. Байконур")
    ]
    print('\tКоличество успешных записей (тип записи "К врачу"; рассылка (дашборд)):')
    print(
        f'\t\t{df_tmp_s[df_tmp_s["Тип записи"] == 3].shape[0]} ({df_tmp_s[df_tmp_s["Тип записи"] == 3].shape[0] - df_nosub_s[df_nosub_s["Тип записи"] == 3].shape[0]})'
    )
    print("\tСубъекты, не отображаемые на дашборде:")
    if df_nosub_s.empty:
        print("\t\tотсутствуют")
    else:
        print(
            df_nosub_s.groupby(["Название субъекта РФ"])["session_id"]
            .count()
            .reset_index()
            .to_string(index=False)
        )
    # break


Файл И38_успешные_сессии_20240127.csv:
		всего строк:		65978
		всего столбцов:		9 (верно)
	Пропуски данных (всего (доля от общего числа строк)):
		Дата:			0	(0.0 %)
		session_id:		0	(0.0 %)
		Название субъекта РФ:	0	(0.0 %)
		sp_oid:			1472	(2.23 %)
		sp_name:		0	(0.0 %)
		post_name:		0	(0.0 %)
		slot_ts:		0	(0.0 %)
		create_ts:		0	(0.0 %)
		Тип записи:		0	(0.0 %)
	Количество успешных записей (тип записи "К врачу"; рассылка (дашборд)):
		65932 (65914)
	Субъекты, не отображаемые на дашборде:
                      Название субъекта РФ  session_id
Федеральное медико-биологическое агентство          18

Файл И38_успешные_сессии_20240128.csv:
		всего строк:		82015
		всего столбцов:		9 (верно)
	Пропуски данных (всего (доля от общего числа строк)):
		Дата:			0	(0.0 %)
		session_id:		0	(0.0 %)
		Название субъекта РФ:	0	(0.0 %)
		sp_oid:			1421	(1.73 %)
		sp_name:		0	(0.0 %)
		post_name:		0	(0.0 %)
		slot_ts:		0	(0.0 %)
		create_ts:		0	(0.0 %)
		Тип записи:		0	(0.0 %)
	Количество успешных запи

In [10]:
df_tmp_s

,Дата,session_id,Название субъекта РФ,sp_oid,sp_name,post_name,slot_ts,create_ts,Тип записи
0,2024-01-28,3693200317,Приморский край,1.2.643.5.1.13.13.12.2.25.2290.0.30302,"""Владивостокская поликлиника №6"", Отделение ме...",врач-хирург,2024-01-29 07:00:00+03:00,2024-01-28 02:26:00.676000+03:00,3
1,2024-01-28,3693134175,Краснодарский край,1.2.643.5.1.13.13.12.2.23.1961.0.312460,"КДО № 3 ГБУЗ ""Центр охраны материнства и детст...",врач-акушер-гинеколог,2024-02-09 15:15:00+03:00,2024-01-28 01:03:15.539000+03:00,3
2,2024-01-28,3693062143,Московская область,1.2.643.5.1.13.13.12.2.50.4653.0.140215,"Взрослая поликлиника ГБУЗ МО ""Электрогорская ГБ""",врач-дерматовенеролог,2024-02-09 08:45:00+03:00,2024-01-28 00:12:58.410000+03:00,3
3,2024-01-28,3693285096,Приморский край,1.2.643.5.1.13.13.12.2.25.2300.0.332259,"""Краевая детская клиническая больница №2"", ул....",врач-педиатр участковый,2024-01-30 07:12:00+03:00,2024-01-28 05:24:51.402000+03:00,3
4,2024-01-28,3693067544,Республика Татарстан,1.2.643.5.1.13.13.12.2.16.1122.0.576562,"Поликлиническое отделение 1, ГАУЗ ""Детская гор...",врач-офтальмолог,2024-02-08 09:40:00+03:00,2024-01-28 00:16:58.224000+03:00,3
...,...,...,...,...,...,...,...,...,...
82010,2024-01-28,3694132918,Хабаровский край,1.2.643.5.1.13.13.12.2.27.2584.0.145066,"Поликлиника Тихоокеанская, 213, КГБУЗ ""ГКБ"" ИМ...",врач-терапевт участковый,2024-02-08 17:45:00+03:00,2024-01-28 22:31:40.396000+03:00,3
82011,2024-01-28,3694138945,Курская область,1.2.643.5.1.13.13.12.2.46.4362.0.42429,"детская поликлиника, ОБУЗ ""Курская горбольница...",врач-педиатр участковый,2024-02-05 17:07:00+03:00,2024-01-28 22:36:05.906000+03:00,3
82012,2024-01-28,3694162691,Московская область,1.2.643.5.1.13.13.12.2.50.49707.0.537144,Взрослая поликлиника № 4 (ГБУЗ МО Красногорска...,врач-терапевт участковый,2024-01-29 09:24:00+03:00,2024-01-28 23:04:25.604000+03:00,3
82013,2024-01-28,3694193463,Хабаровский край,1.2.643.5.1.13.13.12.2.27.2539.0.710328,"Женская консультация № 2, КГБУЗ ""РОДИЛЬНЫЙ ДОМ...",врач-акушер-гинеколог,2024-02-09 19:00:00+03:00,2024-01-28 23:28:17.525000+03:00,3


### Витрины

In [2]:
df_dm = pd.read_excel(
    dm_path + "/" + max(os.listdir(dm_path)) + "/" + dm_file_last[0],
    sheet_name="Сессии по датам",
    index_col=None,
)
print("Файл: ...", max(os.listdir(dm_path)) + "/" + dm_file_last[0])
print(f"\t\tвсего строк:\t\t{df_dm.shape[0]}")
cnt_dm = "верно" if df_dm.shape[1] == 12 else "ошибка"
print(f"\t\tвсего столбцов:\t\t{df_dm.shape[1]} ({cnt_dm})")
print(
    f'\t\tдиапазон дат: \t\t{str(df_dm["Дата"].min())[:10]} - {str(df_dm["Дата"].max())[:10]}'
)
if "Шаг 4. Запись. Уникальные сессии" not in df_dm["Шаг"].unique():
    print("\t\tназвание шага 4:\tотличается от используемого для расчётов")
else:
    print("\t\tназвание шага 4:\t'Шаг 4. Запись. Уникальные сессии' - верное")
# названи шагов из файла
step_name_list = df_dm["Шаг"].unique()
print("\tНазвания шагов, используемые в файле:")
for step_name in step_name_list:
    print("\t\t", step_name)
# пропуски данных
print(
    f'\tПропуски данных за весь период и за {str(df_dm["Дата"].max())[:10]} (всего (доля от общего числа строк)):'
)
missing_dm_dict = {}
missing_dm_lastdate_dict = {}
df_dm_lastdate = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]
for column in df_dm.columns:
    missing_dm_dict[column] = [
        df_dm[column].isnull().sum(),
        round(df_dm[column].isnull().sum() * 100 / df_dm.shape[0], 2),
    ]
    missing_dm_lastdate_dict[column] = [
        df_dm_lastdate[column].isnull().sum(),
        round(df_dm_lastdate[column].isnull().sum() * 100 / df_dm_lastdate.shape[0], 2),
    ]
print(f'\t\t\t\t\tвесь период\t\t{str(df_dm["Дата"].max())[:10]}')

print(
    f'\t\tКод региона:\t\t{missing_dm_dict["Код региона"][0]}\t({missing_dm_dict["Код региона"][1]} %)\t\t{missing_dm_lastdate_dict["Код региона"][0]}\t({missing_dm_lastdate_dict["Код региона"][1]} %)'
)
print(
    f'\t\tРегион:\t\t\t{missing_dm_dict["Регион"][0]}\t({missing_dm_dict["Регион"][1]} %)\t\t{missing_dm_lastdate_dict["Регион"][0]}\t({missing_dm_lastdate_dict["Регион"][1]} %)'
)
print(
    f'\t\tДата:\t\t\t{missing_dm_dict["Дата"][0]}\t({missing_dm_dict["Дата"][1]} %)\t\t{missing_dm_lastdate_dict["Дата"][0]}\t({missing_dm_lastdate_dict["Дата"][1]} %)'
)
print(
    f'\t\tШаг:\t\t\t{missing_dm_dict["Шаг"][0]}\t({missing_dm_dict["Шаг"][1]} %)\t\t{missing_dm_lastdate_dict["Шаг"][0]}\t({missing_dm_lastdate_dict["Шаг"][1]} %)'
)
print(
    f'\t\tВсего:\t\t\t{missing_dm_dict["Всего"][0]}\t({missing_dm_dict["Всего"][1]} %)\t\t{missing_dm_lastdate_dict["Всего"][0]}\t({missing_dm_lastdate_dict["Всего"][1]} %)'
)
print(
    f'\t\tУспешно:\t\t{missing_dm_dict["Успешно"][0]}\t({missing_dm_dict["Успешно"][1]} %)\t\t{missing_dm_lastdate_dict["Успешно"][0]}\t({missing_dm_lastdate_dict["Успешно"][1]} %)'
)
print(
    f'\t\tОшибки ЕПГУ:\t\t{missing_dm_dict["Ошибки ЕПГУ"][0]}\t({missing_dm_dict["Ошибки ЕПГУ"][1]} %)\t\t{missing_dm_lastdate_dict["Ошибки ЕПГУ"][0]}\t({missing_dm_lastdate_dict["Ошибки ЕПГУ"][1]} %)'
)
print(
    f'\t\tТех ошибки:\t\t{missing_dm_dict["Тех ошибки"][0]}\t({missing_dm_dict["Тех ошибки"][1]} %)\t\t{missing_dm_lastdate_dict["Тех ошибки"][0]}\t({missing_dm_lastdate_dict["Тех ошибки"][1]} %)'
)
print(
    f'\t\tОрг ошибки:\t\t{missing_dm_dict["Орг ошибки"][0]}\t({missing_dm_dict["Орг ошибки"][1]} %)\t\t{missing_dm_lastdate_dict["Орг ошибки"][0]}\t({missing_dm_lastdate_dict["Орг ошибки"][1]} %)'
)
print(
    f'\t\tПОДД:\t\t\t{missing_dm_dict["ПОДД"][0]}\t({missing_dm_dict["ПОДД"][1]} %)\t\t{missing_dm_lastdate_dict["ПОДД"][0]}\t({missing_dm_lastdate_dict["ПОДД"][1]} %)'
)
print(
    f'\t\tБрошенные:\t\t{missing_dm_dict["Брошенные"][0]}\t({missing_dm_dict["Брошенные"][1]} %)\t\t{missing_dm_lastdate_dict["Брошенные"][0]}\t({missing_dm_lastdate_dict["Брошенные"][1]} %)'
)
print(
    f'\t\tБизнес-правила:\t\t{missing_dm_dict["Бизнес-правила"][0]}\t({missing_dm_dict["Бизнес-правила"][1]} %)\t{missing_dm_lastdate_dict["Бизнес-правила"][0]}\t({missing_dm_lastdate_dict["Бизнес-правила"][1]} %)'
)
print(
    f'\tРезультаты записи на приём с использованием витрин ЕПГУ на {str(df_dm["Дата"].max())[:10]}'
)
print("\t(бизнес-правила игнорируются):")
# количество успешных записей
dm_s = df_dm[
    (df_dm["Дата"] == df_dm["Дата"].max()) & (df_dm["Шаг"] == step_name_list[3])
]["Успешно"].sum()
# количество технических ошибок
dm_e_t = (
    df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Тех ошибки"].sum()
    + df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["ПОДД"].sum()
)
# количество организационных ошибок
dm_e_o = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Орг ошибки"].sum()
# количество ошибок ФЭР
dm_e_r = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Ошибки ЕПГУ"].sum()
# черновики
dm_e_d = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Брошенные"].sum()
# бизнес-правила
dm_e_b = int(df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Бизнес-правила"].sum())
print(f"\t\tуспешные записи:\t{dm_s}")
print(f"\t\tтехнические ошибки:\t{dm_e_t}")
print(f"\t\tорганизационные ошибки:\t{dm_e_o}")
print(f"\t\tошибки ФЭР:\t\t{dm_e_r}")
print(f"\t\tчерновики:\t\t{dm_e_d}")
print(f"\t\tбизнес-правила:\t\t{dm_e_b}")

# проверка названий регионов
df_dm_regname_ref = pd.read_csv(vis_reg_ref_path, sep=";")
print(
    "Есть в актуальном файле витрин, но нет в чат-боте (не будут распознаны и рассчитаны):"
)
df_dm_regname_actual = pd.DataFrame(df_dm["Регион"].unique().tolist()).rename(
    columns={0: "dm_reg_name_act"}
)
df_dm_regname_contrast = df_dm_regname_actual.merge(
    df_dm_regname_ref, left_on="dm_reg_name_act", right_on="dm_reg_name_ref", how="left"
)
df_dm_regname_contrast_print = df_dm_regname_contrast[
    df_dm_regname_contrast["dm_reg_name_ref"].isnull()
]
if df_dm_regname_contrast_print.empty:
    print(
        f"в файле данных витрин нет регионов, не указанных в справочнике {vis_reg_ref_path}"
    )
else:
    print(df_dm_regname_contrast_print.to_string(index=False))

Файл: ... 2024-01-28/Ежедневные_данные_по_сессиям.xlsx
		всего строк:		26608
		всего столбцов:		12 (верно)
		диапазон дат: 		2023-06-12 - 2024-01-28
		название шага 4:	'Шаг 4. Запись. Уникальные сессии' - верное
	Названия шагов, используемые в файле:
		 Шаг 1. Идентификация пациента в регионе. Уникальные сессии
		 Шаг 2. Выбор специальности. Уникальные сессии
		 Шаг 3. Выбор врача и времени. Уникальные сессии
		 Шаг 4. Запись. Уникальные сессии
	Пропуски данных за весь период и за 2024-01-28 (всего (доля от общего числа строк)):
					весь период		2024-01-28
		Код региона:		0	(0.0 %)		0	(0.0 %)
		Регион:			0	(0.0 %)		0	(0.0 %)
		Дата:			0	(0.0 %)		0	(0.0 %)
		Шаг:			0	(0.0 %)		0	(0.0 %)
		Всего:			0	(0.0 %)		0	(0.0 %)
		Успешно:		0	(0.0 %)		0	(0.0 %)
		Ошибки ЕПГУ:		0	(0.0 %)		0	(0.0 %)
		Тех ошибки:		0	(0.0 %)		0	(0.0 %)
		Орг ошибки:		0	(0.0 %)		0	(0.0 %)
		ПОДД:			0	(0.0 %)		0	(0.0 %)
		Брошенные:		0	(0.0 %)		0	(0.0 %)
		Бизнес-правила:		18308	(68.81 %)	0	(0.0 %)
	Результаты записи н

#### Автоматизированная загрузка списка витринных регионов из последнего файла чат-бота

In [3]:
viz_dir_path = "C:/work/new_bot/new"
viz_file_list = os.listdir(viz_dir_path)
macros_files_dict = {}
for filename in viz_file_list:
    if filename.split(".")[-1] == "xlsm":
        macros_files_dict[os.path.getmtime(viz_dir_path + "/" + filename)] = filename
last_macros_path = viz_dir_path + "/" + macros_files_dict[max(macros_files_dict.keys())]
try:
    df_regname_macros = pd.read_excel(
        last_macros_path,
        sheet_name="СС",
        index_col=None,
    )
except Exception as e:
    sys.exit(e)
print(f"используется файл чат-бота:\t{last_macros_path}")
# список регионов с листа СС макроса
df_dm_regname_macros = (
    pd.DataFrame(df_regname_macros["Unnamed: 4"])
    .dropna()
    .rename(columns={"Unnamed: 4": "macros_reg_name"})
)
# список регионов из моего справочника
df_dm_regname_ref = pd.read_csv(vis_reg_ref_path, sep=";")
print(f"используется справочник:\t{vis_reg_ref_path}")

# список регионов из актуального исходника витрин
df_dm = pd.read_excel(
    dm_path + "/" + max(os.listdir(dm_path)) + "/" + dm_file_last[0],
    sheet_name="Сессии по датам",
    index_col=None,
)
df_dm_regname_actual = pd.DataFrame(df_dm["Регион"].unique().tolist()).rename(
    columns={0: "dm_reg_name_act"}
)
dm_path_used = dm_path + "/" + max(os.listdir(dm_path)) + "/" + dm_file_last[0]
print(f"используется исходник витрин:\t{dm_path_used}")
df_dm_full_reg = df_dm_regname_actual.merge(
    df_dm_regname_macros,
    left_on="dm_reg_name_act",
    right_on="macros_reg_name",
    how="outer",
).merge(
    df_dm_regname_ref,
    left_on="dm_reg_name_act",
    right_on="dm_reg_name_ref",
    how="outer",
)
df_dm_full_reg.sort_values("dm_reg_name_act", inplace=True)
df_dm_full_reg.rename(
    columns={
        "dm_reg_name_act": "есть в исходнике витрин",
        "macros_reg_name": "есть в файле чат-бота",
        "dm_reg_name_ref": "есть в моём справочнике",
    },
    inplace=True,
)
# print(df_dm_full_reg.to_string(index=False))
df_dm_full_reg.reset_index(drop=True)

используется файл чат-бота:	C:/work/new_bot/new/ЭЗ_бот_2024-01-26.xlsm
используется справочник:	C:/work/new_bot/new/vis_reg_ref.csv
используется исходник витрин:	C:/work/new_bot/new/dm/2024-01-28/Ежедневные_данные_по_сессиям.xlsx


,есть в исходнике витрин,есть в файле чат-бота,есть в моём справочнике
0,Астраханская область,Астраханская область,Астраханская область
1,Белгородская область,Белгородская область,Белгородская область
2,Брянская область,Брянская область,Брянская область
3,Владимирская область,Владимирская область,Владимирская область
4,Волгоградская область,Волгоградская область,Волгоградская область
5,Вологодская область,Вологодская область,Вологодская область
6,Воронежская область,Воронежская область,Воронежская область
7,Еврейская автономная область,Еврейская автономная область,Еврейская автономная область
8,Иркутская область,Иркутская область,Иркутская область
9,Кабардино-Балкарская Республика,Кабардино-Балкарская Республика,Кабардино-Балкарская Республика


### Выгрузки ПОС

In [4]:
df_pos_ref_reg = pd.read_csv(pos_reg_ref_path, sep=";")
df_pos = pd.DataFrame()
for file in pos_dir_last:
    df_pos_tmp = pd.read_excel(
        pos_path + "/" + max(os.listdir(pos_path)) + "/" + file,
        sheet_name="Sheet0",
        index_col=None,
    )
    df_pos = pd.concat([df_pos, df_pos_tmp])
    print(f"Загружен файл {file}")
df_pos.reset_index(drop=True, inplace=True)
df_pos_act_reg = pd.DataFrame(df_pos["Верхнеуровневый ЛКО"].unique()).rename(
    columns={0: "actual_reg_name"}
)
df_reg_pos_contrast = df_pos_act_reg.merge(
    df_pos_ref_reg, left_on="actual_reg_name", right_on="pos_reg_name", how="left"
)
no_regnames = df_reg_pos_contrast[df_reg_pos_contrast["pos_reg_name"].isnull()][
    "actual_reg_name"
].tolist()
print(f"Отсутствуют в актуальном списке регионов для расчёта рассылки ({pos_reg_ref_path}):")
for name in no_regnames:
    print("\t", name)

Загружен файл Выгрузка ПОС 01.03.2023 по 31.05.2023.xlsx
Загружен файл Выгрузка ПОС 01.06.2023 по 31.08.2023.xlsx
Загружен файл Выгрузка ПОС 01.09.2023 по 30.11.2023.xlsx
Загружен файл Выгрузка ПОС 01.12.2023 по 28.01.2024.xlsx
Загружен файл Выгрузка ПОС 11.11.2022 по 28.02.2023.xlsx
Отсутствуют в актуальном списке регионов для расчёта рассылки (C:/work/pos_bot/pos_reg_ref.csv):
	 ФЕДЕРАЛЬНОЕ МЕДИКО-БИОЛОГИЧЕСКОЕ АГЕНТСТВО
	 АДМИНИСТРАЦИЯ ГУБЕРНАТОРА ТЕСТОВОГО РЕГИОНА
	 АППАРАТ ПРАВИТЕЛЬСТВА ДОНЕЦКОЙ НАРОДНОЙ РЕСПУБЛИКИ
	 ВОЕННО-ГРАЖДАНСКАЯ АДМИНИСТРАЦИЯ ХЕРСОНСКОЙ ОБЛАСТИ
	 АДМИНИСТРАЦИЯ ГЛАВЫ ЛУГАНСКОЙ НАРОДНОЙ РЕСПУБЛИКИ


***

In [11]:
df_dm_regname_ref

,dm_reg_name_ref,nopass
0,Астраханская область,Астраханская область
1,Белгородская область,Белгородская область
2,Брянская область,Брянская область
3,Владимирская область,Владимирская область
4,Волгоградская область,Волгоградская область
5,Вологодская область,Вологодская область
6,Воронежская область,Воронежская область
7,Еврейская автономная область,Еврейская автономная область
8,Иркутская область,Иркутская область
9,Кабардино-Балкарская Республика,Кабардино-Балкарская Республика


In [10]:
df_dm_regname_ref["nopass"] = df_dm_regname_ref["dm_reg_name_ref"].apply(
    lambda x: " ".join(x.strip().split(" "))
)